In [1]:
import torch.nn
import sympy as sp


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/local/Cellar/python@3.10/3.10.18/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/local/Cellar/python@3.10/3.10.18/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/rthomas/Documents/GitHub/Machine-learning/venv/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  Fi

In [ ]:


#intro

print('Hi! My name is Coniec! Give me a polynomial function and I will try to guess what the output is for an input that you provide - without explicitly calculating it!')
function_input = input('Enter your function here - remember, use strict python syntax: * for multiplication, / for division, ** for exponentiation, + for addition, -for subtraction and coefficients before the variable!-----:')

#geet bounds
print('Enter the upper and lower bounds for x values you would like to give me:')
upper=float(input('Upper bound:'))
lower=float(input('Lower bound:'))


Hi! My name is Coniec! Give me a polynomial function and I will try to guess what the output is for an input that you provide - without explicitly calculating it!


In [ ]:
# make training input values
x_s = torch.arange(lower, upper, 0.11)


#get function from user
x_input = sp.symbols('x')
function = sp.lambdify(x_input, function_input, 'numpy')

# make training output values
y_s = function(x_s)

In [ ]:
#choose number of neurons in layer
n_hidden_1 = 200

#make parameters
W1 = torch.randn(1, n_hidden_1)
b1 = torch.randn(1, n_hidden_1)

W2 = torch.randn(n_hidden_1,1)
b2 = torch.randn(1,1)


parameters = [W1, W2, b1, b2]



for p in parameters:
    p.requires_grad = True


#repeat training 100000 times!
max_steps = 100000

#we calculate loss function of a batch of inputs to avoid overfitting...
batch_size = 32

#we normalise training data and train our multilayer perceptron on this. This stops our loss function from exploding!
x_mean, x_std = x_s.mean(), x_s.std()
y_mean, y_std = y_s.mean(), y_s.std()

x_s_norm = (x_s - x_mean) / x_std
y_s_norm = (y_s - y_mean) / y_std

#define the loss function. This is a regression problem so we use the mean squared error loss function.
loss_fn = torch.nn.MSELoss()





In [ ]:

#begin training
for i in range(max_steps):

    #generate batch indices, i.e. choose 32 random indices we can use to pick out values from our training set
    ix = torch.randint(0,len(x_s), (batch_size,1) )
    Xb, Yb = x_s_norm[ix], y_s_norm[ix]


    #forward pass

    #Linear layer
    hpreact = Xb @ W1 + b1



    #Non-linearity
    h = torch.tanh(hpreact)

    #2nd linear layer
    output = h @ W2 + b2

    #find loss
    mse_loss = loss_fn(Yb, output)


    #backward pass - we calculate gradient of loss function with respect to EVERY SINGLE PARAMETER.
    for p in parameters:
        p.grad=None
    mse_loss.backward()

    #we update every parameter by a small step in the opposite direction to the partial derivative of the loss function with respect to this parameter.
    #This changes each parameter as to minimise the loss function and make our neural network as accurate as possible. A classic step of the backpropagation algorithm.
    lr = 0.01 if i <50000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad


    #print our loss function value every now and then. Ideally it would keep decreasing, but here it fluctuate towards the end. We risk overfitting our neural net
    #to our data. More time would have let me set up batch normalisation for each layer to combat this issue.
    if i %10000 ==0:
        print(f'{i:7d}/{max_steps:7d} training rounds : loss = {mse_loss.item():.4f}')


      0/ 100000 training rounds : loss = 436.3403
  10000/ 100000 training rounds : loss = 0.0002
  20000/ 100000 training rounds : loss = 0.0001
  30000/ 100000 training rounds : loss = 0.0000
  40000/ 100000 training rounds : loss = 0.0000
  50000/ 100000 training rounds : loss = 0.0000
  60000/ 100000 training rounds : loss = 0.0000
  70000/ 100000 training rounds : loss = 0.0000
  80000/ 100000 training rounds : loss = 0.0000
  90000/ 100000 training rounds : loss = 0.0000


In [ ]:

#sample from model
flag=True
while flag==True:
    x_sample = float(input(f'enter an x value between {lower} and {upper} to enter into your function {function_input}: '))
    #our sample is normalised
    x_sample_norm = (x_sample-x_mean)/x_std
    x_sample_norm = torch.tensor([[x_sample_norm]])
    y_sample_norm = torch.tanh(x_sample_norm @ W1 +b1) @ W2 + b2
    #the network output is denormalised before being displayed.
    y_sample =  y_sample_norm*y_std+y_mean
    print('I think the y value is', y_sample.item())
    cont = input(f'The actual value is {function(x_sample)} Was I close?, Type "y" if you want to input another x between {lower} and {upper} or any other key to stop ')
    if cont=='y':
        pass
    else:
        flag=False

enter an x value between 0.0 and 100.0 to enter into your function 9*x**2-3*x: 
I think the y value is 77451.421875
The actual value is 77562.0 Was I close?, Type "y" if you want to input another x between 0.0 and 100.0 or any other key to stop 
